https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

A custom Dataset class must implement three functions: __init__, __len__, and __getitem__.

https://github.com/bomri/SlowFast/blob/master/slowfast/datasets/loader.py

https://github.com/bomri/SlowFast/blob/master/slowfast/datasets/ava_dataset.py

https://github.com/HHTseng/video-classification/blob/master/ResNetCRNN_varylength/UCF101_ResNetCRNN_varlen.py
https://www.ai-contentlab.com/2023/01/video-classification-is-important-task.html

https://discuss.pytorch.org/t/how-upload-sequence-of-image-on-video-classification/24865/13

Оптический поток
https://docs.opencv.org/2.4/modules/video/doc/motion_analysis_and_object_tracking.html

Skeleton
https://www.fireblazeaischool.in/blogs/human-pose-estimation-using-opencv/

# Data Loader

Добавить нормализацию!!!

In [1]:
labels_list = ['пингвин',
 'жираф',
 'лягушка',
 'бегемот',
 'козел',
 'лиса',
 'динозавр',
 'кролик',
 'собака',
 'обезьяна',
 'корова',
 'свинья',
 'медуза',
 'курица',
 'павлин',
 'дельфин',
 'слон',
 'медведь',
 'лебедь',
 'орел',
 'бык',
 'змея',
 'птица',
 'лось',
 'пчела',
 'лев',
 'тигр',
 'мышь',
 'паук',
 'бабочка']

 # Курс Седжвика по алгоритмам

In [2]:
import os
import cv2
import torch
torch.cuda.empty_cache()
from torch import nn
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

In [9]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

def load_video(path, begin, end, base_name, resize=(10, 10)):
    cap = cv2.VideoCapture(path)
    frame_index=begin
    i = 0
    try:
        while True and frame_index <= end:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # convert to grayscale
            frame_name = f'{base_name}_{i}.png'
            cv2.imwrite(frame_name, frame)
            frame_index+=1
            i+=1
    finally:
        cap.release()


In [4]:
annotations_file = "/content/SLOVO_DF_SHORT.tsv"
video_dir = "/content/videos"
dataset_dir = "/content/dataset"
IMG_SIZE = 224
BATCH_SIZE = 6
LEARNING_RATE = 0.001
NUM_EPOCHS = 10

In [5]:
video_labels = pd.read_csv(annotations_file, sep='\t')
video_labels.head(5)

,Unnamed: 0,attachment_id,text,begin,end
0,0,8f4d3be1-3a09-4d76-94ef-f8b1dbfa686b,пингвин,29,100
1,1,4f9e3cb5-b9de-48bc-a51d-875b8fea8e10,пингвин,21,79
2,2,1de7b5b0-ce08-419f-aeed-e7e480da953d,пингвин,7,59
3,3,72f70640-6931-4f57-8c72-a68e48032cfb,пингвин,22,87
4,4,6933a0f1-a0e1-48d8-91be-b445ca6c80ce,пингвин,9,64


In [6]:
labels_cnt = video_labels.groupby('text').size().to_dict()
print(labels_cnt)
# 75 - train, 15 - val, 10 - test

{'бабочка': 20, 'бегемот': 20, 'бык': 20, 'дельфин': 20, 'динозавр': 20, 'жираф': 20, 'змея': 20, 'козел': 20, 'корова': 20, 'кролик': 20, 'курица': 20, 'лебедь': 20, 'лев': 20, 'лиса': 20, 'лось': 20, 'лягушка': 20, 'медведь': 20, 'медуза': 20, 'мышь': 20, 'обезьяна': 20, 'орел': 20, 'павлин': 20, 'паук': 20, 'пингвин': 20, 'птица': 20, 'пчела': 20, 'свинья': 20, 'слон': 20, 'собака': 20, 'тигр': 20}


In [10]:
test_counter = dict()
val_counter = dict()
train_counter = dict()

for idx, row in video_labels.iterrows():
  attachment_id = video_labels.iloc[idx]['attachment_id']
  filename = os.path.join(video_dir, attachment_id+".mp4")
  label = video_labels.iloc[idx]['text']
  begin = video_labels.iloc[idx]['begin']
  end = video_labels.iloc[idx]['end']
  if label not in train_counter:
    dataset_type = 'train'
    train_counter[label] = 1
  elif train_counter[label] < labels_cnt[label]*0.75:
    dataset_type = 'train'
    train_counter[label] += 1
  elif label not in val_counter:
    dataset_type = 'val'
    val_counter[label] = 1
  elif val_counter[label] < val_counter[label]*0.25:
    dataset_type = 'val'
    val_counter[label] += 1
  elif label not in test_counter:
    dataset_type = 'test'
    test_counter[label] = 1
  else:
    dataset_type = 'test'
    test_counter[label] += 1

  base_name = os.path.join(dataset_dir , dataset_type, label, attachment_id)
  load_video(path=filename,
             begin=begin,
             end=end,
             base_name=base_name,
             resize=(10, 10)
             )


In [11]:
os.listdir(dataset_dir)

FileNotFoundError: ignored

In [13]:
base_name

'/content/dataset/test/бабочка/e421d77c-1dcd-4124-b2a4-b2415569a4dd'

In [ ]:
z+1
https://stackoverflow.com/questions/63567352/cnn-rnn-architecture-for-video-recognition
https://discuss.pytorch.org/t/cnn-lstm-implementation-for-video-classification/52018

In [ ]:


class BasicVideoDataset(Dataset):
    def __init__(self, annotations_file, video_dir, IMG_SIZE, labels_list):
        self.video_labels = pd.read_csv(annotations_file, sep='\t')
        self.video_dir = video_dir
        self.IMG_SIZE = IMG_SIZE
        self.frames_cnt = max(self.video_labels['end']-self.video_labels['begin'])
        self.labels_list = labels_list

    def __len__(self):
        return len(self.video_labels)

]




    def __getitem__(self, idx):
        filename  = os.path.join(self.video_dir, self.video_labels.iloc[idx]['attachment_id']+".mp4")
        label = self.video_labels.iloc[idx]['text']
        begin = self.video_labels.iloc[idx]['begin']
        end = self.video_labels.iloc[idx]['end']
        frames = self.load_video(filename, begin, end, resize=(self.IMG_SIZE, self.IMG_SIZE)) # Загрузка видео!!!!
        return frames[: self.frames_cnt], labels_list.index(label)

* The __init__ function is run once when instantiating the Dataset object. We initialize the directory containing the images, the annotations file, and both transforms (covered in more detail in the next section).
* The __len__ function returns the number of samples in our dataset.
* The __getitem__ function loads and returns a sample from the dataset at the given index idx.

In [ ]:
training_data = BasicVideoDataset(annotations_file, video_dir, IMG_SIZE=IMG_SIZE, labels_list=labels_list)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
frames, label = next(iter(train_dataloader))

In [ ]:
frames.shape

torch.Size([6, 110, 224, 224])

In [ ]:
label.shape

torch.Size([6])

# Model

We want to be able to train our model on a hardware accelerator like the GPU or MPS, if available. Let’s check to see if torch.cuda or torch.backends.mps are available, otherwise we use the CPU.

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")
#device = "cuda"

Using cuda device


We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. Every nn.Module subclass implements the operations on input data in the forward method.

In [ ]:
class cnn2d(nn.Module):
    def __init__(self, IMG_SIZE=224):
        super(cnn2d, self).__init__()
        self.IMG_SIZE = IMG_SIZE
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=110, out_channels=50, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.layer2 = nn.Sequential(
            nn.Conv2d(50, 25, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.layer3 = nn.Sequential(
            nn.Conv2d(25, 5, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(5*28*28, 1000)
        self.fc2 = nn.Linear(1000, 30)

    def forward(self, x):
     out = self.layer1(x)
     out = self.layer2(out)
     out = self.layer3(out)
     out = out.reshape(out.size(0), -1)
     out = self.drop_out(out)
     out = self.fc1(out)
     out = self.fc2(out)
     return out

     # https://neurohive.io/ru/tutorial/cnn-na-pytorch/


We create an instance of NeuralNetwork, and move it to the device, and print its structure.

In [ ]:
model = cnn2d().to(device)
print(model)

cnn2d(
  (layer1): Sequential(
    (0): Conv2d(110, 50, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(50, 25, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(25, 5, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=3920, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=30, bias=True)
)


In [ ]:
input = torch.randn(BATCH_SIZE, 110, IMG_SIZE, IMG_SIZE).to(device)
output = model(input)
output.shape # [4, 8, 56, 56]

torch.Size([6, 30])

In [ ]:
8*56*56

25088

To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.

# Обучение

In [ ]:
model = cnn2d().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
total_step = len(train_dataloader)
loss_list = []
acc_list = []
for epoch in range(NUM_EPOCHS):
    for i, (frames, labels) in enumerate(train_dataloader):
      frames = frames.to(device, dtype=torch.float)
      labels = labels.to(device, dtype=torch.float)
       # Прямой запуск
      outputs = model(frames)
      labels=labels.to(torch.int64)
      loss = criterion(outputs, labels)
      loss_list.append(loss.item())

        # Обратное распространение и оптимизатор
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

        # Отслеживание точности
      total = labels.size(0)
      _, predicted = torch.max(outputs.data, 1)
      correct = (predicted == labels).sum().item()
      acc_list.append(correct / total)

      if (i + 1) % 10 == 0:
          print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, NUM_EPOCHS, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/10], Step [10/100], Loss: 3.3898, Accuracy: 0.00%
Epoch [1/10], Step [20/100], Loss: 3.3922, Accuracy: 0.00%
Epoch [1/10], Step [30/100], Loss: 3.4089, Accuracy: 0.00%
Epoch [1/10], Step [40/100], Loss: 3.3941, Accuracy: 0.00%
Epoch [1/10], Step [50/100], Loss: 3.4020, Accuracy: 0.00%
Epoch [1/10], Step [60/100], Loss: 3.4209, Accuracy: 0.00%
Epoch [1/10], Step [70/100], Loss: 3.4279, Accuracy: 0.00%
Epoch [1/10], Step [80/100], Loss: 3.4350, Accuracy: 0.00%
Epoch [1/10], Step [90/100], Loss: 3.4053, Accuracy: 0.00%
Epoch [1/10], Step [100/100], Loss: 3.4348, Accuracy: 0.00%
Epoch [2/10], Step [10/100], Loss: 3.3981, Accuracy: 0.00%
Epoch [2/10], Step [20/100], Loss: 3.4015, Accuracy: 16.67%
Epoch [2/10], Step [30/100], Loss: 3.4158, Accuracy: 0.00%
Epoch [2/10], Step [40/100], Loss: 3.3921, Accuracy: 0.00%
Epoch [2/10], Step [50/100], Loss: 3.3860, Accuracy: 0.00%


In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i, (frames, labels) in enumerate(train_dataloader):
      frames = frames.to(device, dtype=torch.float)
      labels = labels.to(device, dtype=torch.float)
      outputs = model(frames)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))



In [ ]:
# # Сохраняем модель и строим график
# torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')

In [ ]:
images.dtype